### Generate Dependency Graph for Numpy, Max Depth = 4

In [1]:
from pygraas import DependencyGraph, VulnerabilityGraph
g = DependencyGraph(package_name="numpy", package_url="https://github.com/numpy/numpy")
graph = g.build_graph(max_bacon=4)

Cloning into 'numpy'...
Updating files: 100% (2222/2222), done.
dummymodule.py:140: WARNING: SKIPPING ILLEGAL MODULE_NAME: numpy.numpy._pyinstaller.hook-numpy
dummymodule.py:140: WARNING: SKIPPING ILLEGAL MODULE_NAME: numpy.numpy._pyinstaller.tests.pyinstaller-smoke



changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
Graph built with 292 nodes and 1494 edges.


### Generate Vulnerability Graph based on the Dependency Graph

Vulnerability Graph marks the external packages which are Vulnerable, also add's the CVEs, advisory and 
vulnerable package versions as node attributes.

In [2]:
v = VulnerabilityGraph(g)
v.build_vulnerability_graph()

#### List external vulnerable packages based on the Safety-DB Database

In [3]:
vulnerables = v.get_vulnerables()
print(vulnerables)

['numpy', 'psutil', 'setuptools']


### Read Details

In [26]:
v.get_vulnerables(details=True)[2]["setuptools"]["advisory"]

['Setuptools version 0.7 includes a fix for CVE-2013-1633: Easy_install in setuptools before 0.7 uses HTTP to retrieve packages from the PyPI repository, and does not perform integrity checks on package contents, which allows man-in-the-middle attackers to execute arbitrary code via a crafted response to the default use of the product.',
 'setuptools 0.9.5 fixes a security vulnerability in SSL certificate validation.',
 'Setuptools before 1.3 has a security vulnerability in SSL match_hostname check, similar to CVE-2013-7440.',
 'setuptools 3.0 avoids the potential security vulnerabilities presented by use of tar archives in ez_setup.py. It also leverages the security features added to ZipFile.extract in Python 2.7.4.',
 'Setuptools 65.5.1 includes a fix for CVE-2022-40897: Python Packaging Authority (PyPA) setuptools before 65.5.1 allows remote attackers to cause a denial of service via HTML in a crafted package or custom PackageIndex page. There is a Regular Expression Denial of Servi

In [27]:
v.get_vulnerables(details=True)[2]["setuptools"]["version"]

['<0.7', '<0.9.5', '<1.3', '<3.0', '<65.5.1', '<70.0.0', '>=0,<0.9.5']

In [29]:
v.get_vulnerables(details=True)[2]["setuptools"]["CVE"]

['CVE-2013-1633',
 'PVE-2021-26129',
 'PVE-2021-26132',
 'PVE-2021-26133',
 'CVE-2022-40897',
 'CVE-2024-6345',
 'PVE-2024-99866']

#### Degree of the "psutil" node 

In [4]:
v.get_degree(vulnerables[1])

{'in_degree': 10, 'out_degree': 1}

#### List of Public Nodes that have a path to vulnerable "setuptools"

In [18]:
# Show only the first 20 entries for demonstration.
v.get_vulnerable_public_nodes(vulnerables[2])[0: 20]

[{'grp - setuptools': True},
 {'email - setuptools': True},
 {'math - setuptools': True},
 {'importlib - setuptools': True},
 {'http - setuptools': True},
 {'collections - setuptools': True},
 {'bisect - setuptools': True},
 {'functools - setuptools': True},
 {'mimetypes - setuptools': True},
 {'fnmatch - setuptools': True},
 {'copy - setuptools': True},
 {'ntpath - setuptools': True},
 {'glob - setuptools': True},
 {'doctest - setuptools': True},
 {'enum - setuptools': True},
 {'codecs - setuptools': True},
 {'genericpath - setuptools': True},
 {'gc - setuptools': True},
 {'fcntl - setuptools': True},
 {'dataclasses - setuptools': True}]